#### Primary Postprocessing Suite

Figures from the primary data $\{\psi, \textbf{u}, c, s\}$ and, if available, trivial secondary data $\{\min_{\textbf{x}}|\textbf{u}|, \max_{\textbf{x}}|\textbf{u}| ,\min_{\textbf{x}}(c), \max_{\textbf{x}}(c), m_D, m_C, \dots \}$ 

In [ ]:
%load_ext autoreload

%autoreload 2

from lucifex.fdm import FunctionSeries, GridSeries, ConstantSeries, NumericSeries
from lucifex.viz.utils import detex
from lucifex.io import find_dataset, find_by_parameters, proxy
from co2_dissolution_pkg.post.plot import (
    plot_colormaps, plot_timeseries, 
    plot_twinned_timeseries, plot_colormaps_multifigure,
)


DATASET = find_dataset('./data/', include='Ra=*')
PARAMETERS = 'PARAMETERS.txt'
FUNCTION_SERIES = 'FunctionSeries.h5'
CONSTANT_SERIES = 'ConstantSeries.h5'
GRID_SERIES = 'GridSeries.npz'
NUMERIC_SERIES = 'NumericSeries.npz'
FIGURES = 'figures'

Ra = 600
Da = 500
sr = 0.1 
parameters = dict(Ra=Ra, Da=Da, sr=sr)
directory = find_by_parameters(DATASET.directories, PARAMETERS, parameters)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
DATASET

In [ ]:
ALL = False
if ALL:
    dir_arg = DATASET.directories
else:
    dir_arg = directory

#### Exploratory Postprocessing

In [ ]:
# mass
fig_names = ('mD(t)', 'mC(t)', 'mCmD(t)')
plot_twinned_timeseries(dir_arg, fig_names, FIGURES)(
    proxy([('mD', NumericSeries, NUMERIC_SERIES), ('mC', NumericSeries, NUMERIC_SERIES)]),
    labels=('m_D', 'm_C'),
)

#### Established Postprocessing

In [ ]:
# utilities
make_suptitle = lambda Ra, Da, sr: f'$(Ra, Da, s_r)=({Ra}, {Da}, {sr})$'

In [ ]:
# mosaic of concentration across times
times = (0.0, 2.0, 4.0, 8.0, 10.0, 15.0, 30.0, 60.0, 80.0)
n_cols = 3
n_rows = len(times) // n_cols
colorbar = (0, 1)
fig_name= f"c(x,y,t={times})_{n_cols}x{n_rows}_colorbar={colorbar}".replace(' ','')
plot_colormaps_multifigure(DATASET.directories, fig_name, FIGURES)(
    proxy(('c', GridSeries, GRID_SERIES)),
    times,
    colorbar,
    n_cols,
    proxy(make_suptitle, [('Ra', float, PARAMETERS), ('Da', float, PARAMETERS), ('sr', float, PARAMETERS)]),
    figscale=1.0,
)

In [ ]:
# mosaic of saturations across times
times = (0.0, 2.0, 4.0, 8.0, 10.0, 15.0, 30.0, 60.0, 80.0)
n_cols = 3
n_rows = len(times) // n_cols
make_colorbar = lambda sr: (0, sr)
make_ylims = lambda h0, Ly: (h0, Ly)
fig_name= f"s(x,y,t={times})_{n_cols}x{n_rows}_colorbar=(0,sr)".replace(' ','')
plot_colormaps_multifigure(DATASET.directories, fig_name, FIGURES)(
    proxy(('s', GRID_SERIES)),
    times,
    proxy(make_colorbar, ('sr', float, PARAMETERS)),
    n_cols,
    proxy(make_suptitle, [('Ra', float, PARAMETERS), ('Da', float, PARAMETERS), ('sr', float, PARAMETERS)]),
    figscale=1.0,
    y_lims=proxy(make_ylims, [('h0', float, PARAMETERS), ('Ly', float, PARAMETERS)]),
    aspect='auto',
)

In [ ]:
# maximum and minimum velocity
fig_names = ('umin(t)', 'umax(t)', 'uminmax(t)')
plot_twinned_timeseries(dir_arg, fig_names, FIGURES)(
    proxy([('umin', NumericSeries, NUMERIC_SERIES), ('umax', NumericSeries, NUMERIC_SERIES)]), 
    labels=('\\min_{\mathbf{x}}|\mathbf{u}|', '\\max_{\mathbf{x}}|\mathbf{u}|'),
)

In [ ]:
# concentration extrema
fig_names = ('cmin(t)', 'cmax(t)', 'cminmax(t)', 'cminmax(t)_twins')
plot_twinned_timeseries(dir_arg, fig_names, FIGURES)(
    proxy([('umin', NumericSeries, NUMERIC_SERIES), ('umax', NumericSeries, NUMERIC_SERIES)]), 
    labels=('\\min_{\mathbf{x}}(c)', '\\max_{\mathbf{x}}(c)'),
    twins=True,
)

In [ ]:
# concentration
fig_title = lambda fig: detex(fig.axes[0].get_title())
fig_name = lambda fig: '_'.join((fig_title(fig), 'colorbar=abs'))
times = (0, 1.0, 3.0, 5.0, 10.0, 20.0, 30.0, 50.0, 100.0)
plot_colormaps(dir_arg, fig_name, FIGURES)(
    proxy(('c', GridSeries, GRID_SERIES)), 
    10,
    colorbar=(0,1),
)
fig_name = lambda fig: '_'.join((fig_title(fig), 'colorbar=rel'))
plot_colormaps(dir_arg, fig_name, FIGURES)(
    proxy(('c', GridSeries, GRID_SERIES)), 
    10,
)

In [ ]:
# saturation
make_ylims = lambda h0, Ly: (h0, Ly)
make_colorbar = lambda Sr: (0, Sr)
fig_name = lambda fig: '_'.join((fig_title(fig), 'colorbar=abs'))
plot_colormaps(dir_arg, fig_name, FIGURES)(
    proxy(('s', GRID_SERIES)), 
    10, 
    colorbar=proxy(make_colorbar, ('Sr', float, PARAMETERS)),
    y_lims=proxy(make_ylims, [('h0', float, PARAMETERS), ('Ly', float, PARAMETERS)]),
    aspect='auto',
)

In [ ]:
# flux
get_fdS_h0_sum = lambda fdS: NumericSeries([sum(i[:3]) for i in fdS.series], fdS.time_series)
plot_timeseries(DATASET.directories, FIGURES)(
    proxy(get_fdS_h0_sum, ('fdS', NumericSeries, NUMERIC_SERIES)),
    'F_{y=h_0}' 
)
get_fdS_mid_sum = lambda fdS: NumericSeries([sum(i[3:]) for i in fdS.series], fdS.time_series)
plot_timeseries(DATASET.directories, FIGURES)(
    proxy(get_fdS_mid_sum, ('fdS', NumericSeries, NUMERIC_SERIES)),
    'F_{y=0.5}' 
)

In [ ]:
# # explicit looping with manual loading
# # advantages: greater flexibility?

# from lucifex.fdm import NumericSeries
# from lucifex.io import load_grid_series, load_numeric_series


# for directory in dataset.directories:
#     fdS = load_numeric_series('fdS', directory, NUMERIC_SERIES)
#     fdS_h0_sum = NumericSeries([sum(i[:3]) for i in fdS.series], fdS.time_series)
#     fdS_mid_sum = NumericSeries([sum(i[3:]) for i in fdS.series], fdS.time_series)
#     fig_ax = ...